In [ ]:
#If you're manually copy-pasting the total profit reported by gurobi and then want to calculate optimality gap, run this cell
# import os
# import pandas as pd

# def calculate_optimality_gap(folder_path):
#     for filename in os.listdir(folder_path):
#         if filename.endswith(".csv"):
#             file_path = os.path.join(folder_path, filename)
#             try:
#                 df = pd.read_csv(file_path)
#                 if "gurobi" in df.columns and "Total Profit" in df.columns:
#                     df["optimality_gap"] = ((df["gurobi"] - df["Total Profit"]).abs() / df["gurobi"].replace(0, 1e-8)) * 100
#                     df.to_csv(file_path, index=False)
#                 else:
#                     print(f"Missing columns: {filename}")
#             except Exception as e:
#                 print(f"Error {filename}: {e}")

# # Specify the folder path containing all the csv's to be processed
# folder = "folder_path"
# calculate_optimality_gap(folder)


In [ ]:
#If you want to read the total_profit reported by Gurobi from the output file of gurobi itself and from there you want to calculte the optimality gap , run this cell
import os
import pandas as pd

def calculate_optimality_gap(folder_path):
    # For renaming gurobi.csv’s profit column
    gurobi_path = os.path.join(folder_path, "gurobi.csv")
    if not os.path.isfile(gurobi_path):
        raise FileNotFoundError(f"Could not find gurobi.csv in {folder_path}")
    df_g = pd.read_csv(gurobi_path)
    if "Total Profit" in df_g.columns:
        df_g = df_g.rename(columns={"Total Profit": "gurobi_profit"})
    elif "total_profit" in df_g.columns:
        df_g = df_g.rename(columns={"total_profit": "gurobi_profit"})
    else:
        raise KeyError("gurobi.csv must have columns either 'Total Profit' or 'total_profit'")
    keys = ["number_of_elements", "capacity"]
    if not set(keys + ["gurobi_profit"]).issubset(df_g.columns):
        missing = set(keys + ["gurobi_profit"]) - set(df_g.columns)
        raise ValueError(f"gurobi.csv: missing columns: {missing}")

    # Iterate over each algorithm CSV
    for fname in os.listdir(folder_path):
        if not fname.endswith(".csv") or fname == "gurobi.csv":
            continue
        algo_path = os.path.join(folder_path, fname)
        df_a = pd.read_csv(algo_path)
        # rename its profit column
        if "Total Profit" in df_a.columns:
            df_a = df_a.rename(columns={"Total Profit": "algo_profit"})
        elif "total_profit" in df_a.columns:
            df_a = df_a.rename(columns={"total_profit": "algo_profit"})
        else:
            print(f"Skipping {fname}: no profit column found")
            continue
        if not set(keys + ["algo_profit"]).issubset(df_a.columns):
            print(f"Skipping {fname}: missing one of {keys + ['algo_profit']}")
            continue

        # Merge on number_of_elements & capacity
        df_merged = pd.merge(
            df_a,
            df_g[keys + ["gurobi_profit"]],
            on=keys,
            how="left"
        )

        df_merged["optimality_gap"] = ((df_merged["gurobi_profit"] - df_merged["algo_profit"]).abs() / df_merged["gurobi_profit"].replace(0, 1e-8)) * 100

        #Overwrite the algorithm CSV
        df_merged.to_csv(algo_path, index=False)
        print(f"Done {fname}")

# # Specify the folder path containing all the csv's to be processed
folder = "/folder"
calculate_optimality_gap(folder)
